<center>
<img src="https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/img/ods_stickers.jpg" />
    
## [mlcourse.ai](https://mlcourse.ai) - Open Machine Learning Course

<center>
Auteur: [Yury Kashnitskiy](https://yorko.github.io). Traduit par Anna Larionova et [Ousmane Cissé](https://fr.linkedin.com/in/ousmane-cisse).  
Ce matériel est soumis aux termes et conditions de la licence [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/).  
L'utilisation gratuite est autorisée à des fins non commerciales.

# <center> Sujet 6. Régression</center>
## <center>Lasso et Regression Ridge</center>

*Le programme du cours diffère cette semaine du plan de l'article, car le sujet 4 (modèles linéaires) est trop vaste et important, nous couvrons donc la régression cette semaine.*

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

%config InlineBackend.figure_format = 'retina'
import seaborn as sns

sns.set()  # just to use the seaborn theme


from sklearn.datasets import load_boston
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn.model_selection import KFold, cross_val_score

**Nous travaillerons avec les données sur les prix des logements à Boston (référentiel UCI).**
**Téléchargez les données.**

In [ ]:
boston = load_boston()
X, y = boston["data"], boston["target"]

**Description des données:**

In [ ]:
print(boston.DESCR)

In [ ]:
boston.feature_names

**Regardons les deux premiers enregistrements.**

In [ ]:
X[:2]

## Régression Lasso

La régression Lasso minimise l'erreur quadratique moyenne avec la régularisation L1:
$$\Large error(X, y, w) = \frac{1}{2} \sum_{i=1}^\ell {(y_i - w^Tx_i)}^2 + \alpha \sum_{i=1}^d |w_i|$$

où l'équation d'hyperplan $y = w^Tx$ en fonction des paramètres du modèle $w$, $\ell$ est le nombre d'observations dans les données $X$, $d$ est le nombre d'entités, valeurs cibles $y$, coefficient de régularisation $\alpha$.

**Ajustons la régression Lasso avec le petit coefficient $\alpha$ (régularisation faible). Le coefficient lié à la caractéristique NOX (concentration en oxydes nitriques) sera nul. Cela signifie que cette caractéristique est la moins importante pour la prévision des prix médians des logements dans cette région.**

In [ ]:
lasso = Lasso(alpha=0.1)
lasso.fit(X, y)
lasso.coef_

**Entraînons la régression Lasso avec $\alpha=10$. Tous les coefficients sont égaux à zéro, à l'exception des caractéristiques ZN (proportion de terrains résidentiels zonés pour des lots de plus de 25 000 pieds carrés), TAXE (taux de taxe foncière de pleine valeur), B (proportion de Noirs par ville) et LSTAT (% de statut inférieur de la population).**

In [ ]:
lasso = Lasso(alpha=10)
lasso.fit(X, y)
lasso.coef_

**Cela signifie que la régression Lasso peut servir de méthode de sélection des caractéristiques.**

In [ ]:
n_alphas = 200
alphas = np.linspace(0.1, 10, n_alphas)
model = Lasso()

coefs = []
for a in alphas:
    model.set_params(alpha=a)
    model.fit(X, y)
    coefs.append(model.coef_)

plt.rcParams["figure.figsize"] = (12, 8)

ax = plt.gca()
# ax.set_color_cycle(['b', 'r', 'g', 'c', 'k', 'y', 'm'])

ax.plot(alphas, coefs)
ax.set_xscale("log")
ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
plt.xlabel("alpha")
plt.ylabel("weights")
plt.title("Lasso coefficients as a function of the regularization")
plt.axis("tight")
plt.show();

**Maintenant, trouvons la meilleure valeur de $\alpha$ lors de la validation croisée.**

In [ ]:
lasso_cv = LassoCV(alphas=alphas, cv=3, random_state=17)
lasso_cv.fit(X, y)

In [ ]:
lasso_cv.coef_

In [ ]:
lasso_cv.alpha_

**Dans Scikit-learn, les métriques sont généralement * maximisées *, donc pour MSE il y a une solution: `neg_mean_squared_error` est plutôt minimisé. Pas vraiment pratique.**

In [ ]:
cross_val_score(Lasso(lasso_cv.alpha_), X, y, cv=3, scoring="neg_mean_squared_error")

In [ ]:
abs(
    cross_val_score(
        Lasso(lasso_cv.alpha_), X, y, cv=3, scoring="neg_mean_squared_error"
    ).mean()
)

In [ ]:
abs(np.mean(cross_val_score(Lasso(9.95), X, y, cv=3, scoring="neg_mean_squared_error")))

**Encore un point ambigu: LassoCV trie les valeurs des paramètres par ordre décroissant pour faciliter l'optimisation. Il peut sembler que l’optimisation $\alpha$ ne fonctionne pas correctement.**

In [ ]:
lasso_cv.alphas[:10]

In [ ]:
lasso_cv.alphas_[:10]

In [ ]:
plt.plot(lasso_cv.alphas, lasso_cv.mse_path_.mean(1))  # incorrect
plt.axvline(lasso_cv.alpha_, c="g");

In [ ]:
plt.plot(lasso_cv.alphas_, lasso_cv.mse_path_.mean(1))  # correct
plt.axvline(lasso_cv.alpha_, c="g");

## Régression Ridge

La régression Ridge minimise l'erreur quadratique moyenne avec la régularisation L2:
$$\Large error(X, y, w) = \frac{1}{2} \sum_{i=1}^\ell {(y_i - w^Tx_i)}^2 + \alpha \sum_{i=1}^d w_i^2$$

où l'équation d'hyperplan $y = w^Tx$ en fonction des paramètres du modèle $w$, $\ell$ est le nombre d'observations dans les données $X$, $d$ est le nombre d'entités, valeurs cibles $y$, coefficient de régularisation $\alpha$.

Il existe une classe spéciale [RidgeCV](http://scikit-learn.org/stable/modules/generated/sklearn.linear _model.RidgeCV.html # sklearn.linear_ model.RidgeCV) pour la régression Ridge validation croisée.

In [ ]:
n_alphas = 200
ridge_alphas = np.logspace(-2, 6, n_alphas)

In [ ]:
ridge_cv = RidgeCV(alphas=ridge_alphas, scoring="neg_mean_squared_error", cv=3)
ridge_cv.fit(X, y)

In [ ]:
ridge_cv.alpha_

**En cas de régression Ridge, aucun des paramètres ne se réduit à zéro. La valeur peut être petite mais non nulle.**

In [ ]:
ridge_cv.coef_

In [ ]:
n_alphas = 200
ridge_alphas = np.logspace(-2, 6, n_alphas)
model = Ridge()

coefs = []
for a in ridge_alphas:
    model.set_params(alpha=a)
    model.fit(X, y)
    coefs.append(model.coef_)

ax = plt.gca()
# ax.set_color_cycle(['b', 'r', 'g', 'c', 'k', 'y', 'm'])

ax.plot(ridge_alphas, coefs)
ax.set_xscale("log")
ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
plt.xlabel("alpha")
plt.ylabel("weights")
plt.title("Ridge coefficients as a function of the regularization")
plt.axis("tight")
plt.show()

## Références
- [Generalized linear models](http://scikit-learn.org/stable/modules/linear_model.html) (Generalized Linear Models, GLM) in Scikit-learn
- [LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression), [Lasso](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso), [LassoCV](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html#sklearn.linear_model.LassoCV), [Ridge](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) and [RidgeCV](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html#sklearn.linear_model.RidgeCV) in Scikit-learn